In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numba as nb

from utils import (
    combine_uncertaintes,
    plot_pixels,
    confidence_interval,
    wald_uncertainty,
)

# ignore deprecation warnings from numba for now
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=NumbaPendingDeprecationWarning)

In [ ]:

from numba import cuda
import numpy as np

@cuda.jit
def is_in_mandelbrot(x, y, output):
    """CUDA kernel to check if point (x, y) is in the Mandelbrot set."""
    i = cuda.grid(1)
    if i < x.size:
        c = np.complex64(x[i]) + np.complex64(y[i]) * np.complex64(1j)
        z_hare = z_tortoise = np.complex64(0)
        while True:
            z_hare = z_hare * z_hare + c
            z_hare = z_hare * z_hare + c  # hare does one step more
            z_tortoise = z_tortoise * z_tortoise + c  # tortoise is one step behind
            if z_hare == z_tortoise:
                output[i] = True
                break
            if z_hare.real**2 + z_hare.imag**2 > 4:
                output[i] = False
                break


In [ ]:

from numba import cuda
import numpy as np

@cuda.jit
def draw_mandelbrot(num_x, num_y, xmin, xmax, ymin, ymax, pixels):
    """Generate Mandelbrot set inside Knill limits using CUDA."""
    i, j = cuda.grid(2)
    if i < num_x and j < num_y:
        x = xmin + i * (xmax - xmin) / num_x
        y = ymin + j * (ymax - ymin) / num_y
        c = x + y * 1j
        z_hare = z_tortoise = 0 + 0j
        while True:
            z_hare = z_hare * z_hare + c
            z_hare = z_hare * z_hare + c
            z_tortoise = z_tortoise * z_tortoise + c
            if z_hare == z_tortoise:
                pixels[i, j] = 1
                break
            if z_hare.real**2 + z_hare.imag**2 > 4:
                pixels[i, j] = 0
                break

# Define constants for the limits
xmin, xmax = -2, 1
ymin, ymax = -3 / 2, 3 / 2

# Setup and run the CUDA kernel
num_x, num_y = 1024, 1024
pixels = np.zeros((num_x, num_y), dtype=np.int32)

threadsperblock = (16, 16)
blockspergrid_x = (num_x + threadsperblock[0] - 1) // threadsperblock[0]
blockspergrid_y = (num_y + threadsperblock[1] - 1) // threadsperblock[1]
blockspergrid = (blockspergrid_x, blockspergrid_y)

draw_mandelbrot[blockspergrid, threadsperblock](num_x, num_y, xmin, xmax, ymin, ymax, pixels)

In [ ]:
pixels = draw_mandelbrot(1000, 1000)
fig, _, _ = plot_pixels(pixels, dpi=80)

In [ ]:
@nb.jit
def count_mandelbrot(rng, num_samples, xmin, width, ymin, height):
    """Draw num_samples random numbers uniformly between (xmin, xmin+width)
    and (ymin, ymin+height).
    Raise `out` by one if the number is part of the Mandelbrot set.
    """
    out = np.int32(0)
    for x_norm, y_norm in rng.random((num_samples, 2), np.float32):
        x = xmin + (x_norm * width)
        y = ymin + (y_norm * height)
        out += is_in_mandelbrot(x, y)
    return out

In [ ]:
# Knill limits
xmin, xmax = -2, 1
ymin, ymax = -3 / 2, 3 / 2

rng = np.random.default_rng()  # can be forked to run multiple rngs in parallel

denominator = 100000  # how many random numbers to draw


numerator = count_mandelbrot(rng, denominator, xmin, xmax - xmin, ymin, ymax - ymin)

# ratio of numbers inside Mandelbrot set times sampling area
area = (numerator / denominator) * (xmax - xmin) * (ymax - ymin)
area

In [ ]:
confidence_interval(0.05, numerator, denominator, (xmax - xmin) * (ymax - ymin))

In [ ]:
region1 = {"xmin": -1.5, "ymin": 0.5, "width": 0.5, "height": 0.5}
region2 = {"xmin": -0.4, "ymin": 0.5, "width": 0.5, "height": 0.5}
region3 = {"xmin": -0.4, "ymin": -0.25, "width": 0.5, "height": 0.5}

for region in [region1, region2, region3]:
    denominator = 10000
    numerator = count_mandelbrot(rng, denominator, region["xmin"], region["width"], region["ymin"], region["height"])

    low, high = confidence_interval(0.05, numerator, denominator, region["width"] * region["height"])

    print(f"{numerator:5d}/{denominator}  -->  low: {low:8.3g}, high: {high:8.3g}  -->  uncertainty: {high - low:8.3g}")

In [ ]:
fig, ax, _ = plot_pixels(pixels, dpi=80)

ax.add_patch(matplotlib.patches.Rectangle((-1.5, 0.5), 0.5, 0.5, edgecolor="red", facecolor="none"))
ax.add_patch(matplotlib.patches.Rectangle((-0.4, 0.5), 0.5, 0.5, edgecolor="red", facecolor="none"))
ax.add_patch(matplotlib.patches.Rectangle((-0.4, -0.25), 0.5, 0.5, edgecolor="red", facecolor="none"))

None

In [ ]:
NUM_TILES_1D = 100

numer = np.zeros((NUM_TILES_1D, NUM_TILES_1D), dtype=np.int64)
denom = np.zeros((NUM_TILES_1D, NUM_TILES_1D), dtype=np.int64)

In [ ]:
width = 3 / NUM_TILES_1D
height = 3 / NUM_TILES_1D

In [ ]:
@nb.jit
def xmin(j):
    return -2 + width * j

@nb.jit
def ymin(i):
    return -3/2 + height * i

In [ ]:
@nb.jit
def compute_sequentially(rng, numer, denom):
    """Sample 100 points in each tile."""
    
    for i in range(NUM_TILES_1D):
        for j in range(NUM_TILES_1D):
            denom[i, j] = 100
            numer[i, j] = count_mandelbrot(rng, denom[i, j], xmin(j), width, ymin(i), height)

compute_sequentially(rng, numer, denom)

In [ ]:
fig, ax, p = plot_pixels(numer / denom, dpi=80)
fig.colorbar(p, ax=ax, shrink=0.8, label="fraction of sampled points in Mandelbrot set in each tile")

None

In [ ]:
rngs = rng.spawn(NUM_TILES_1D * NUM_TILES_1D)

In [ ]:
@nb.jit(parallel=True)
def compute_parallel(rngs, numer, denom):
    """Sample all tiles in parallel with NUM_TILES_1D**2 rngs."""
    for i in nb.prange(NUM_TILES_1D):
        for j in nb.prange(NUM_TILES_1D):
            rng = rngs[NUM_TILES_1D * i + j]  # get rng for this tile

            denom[i, j] = 100
            numer[i, j] = count_mandelbrot(
                rng, denom[i, j], xmin(j), width, ymin(i), height
            )

numer = np.zeros((NUM_TILES_1D, NUM_TILES_1D), dtype=np.int64)
denom = np.zeros((NUM_TILES_1D, NUM_TILES_1D), dtype=np.int64)

compute_parallel(rngs, numer, denom)

In [ ]:
(denom == 100).all()

In [ ]:
CONFIDENCE_LEVEL = 0.05

confidence_interval_low, confidence_interval_high = confidence_interval(
    CONFIDENCE_LEVEL / 2, numer, denom, width * height
)

In [ ]:
fig, ax, p = plot_pixels(confidence_interval_high - confidence_interval_low, dpi=80)
fig.colorbar(p, ax=ax, shrink=0.8, label="size of 95% confidence interval (in units of area) of each tile")

None

In [ ]:
SAMPLES_IN_BATCH = 100

@nb.jit(parallel=True)
def compute_until(rngs, numer, denom, uncert, uncert_target):
    """Compute area of each tile until uncert_target is reached.
    The uncertainty is calculate with the Wald approximation in each tile.
    """
    for i in nb.prange(NUM_TILES_1D):
        for j in nb.prange(NUM_TILES_1D):
            rng = rngs[NUM_TILES_1D * i + j]

            uncert[i, j] = np.inf

            # Sample SAMPLES_IN_BATCH more points until uncert_target is reached
            while uncert[i, j] > uncert_target:
                denom[i, j] += SAMPLES_IN_BATCH
                numer[i, j] += count_mandelbrot(
                    rng, SAMPLES_IN_BATCH, xmin(j), width, ymin(i), height
                )

                uncert[i, j] = (
                    wald_uncertainty(numer[i, j], denom[i, j]) * width * height
                )

numer = np.zeros((NUM_TILES_1D, NUM_TILES_1D), dtype=np.int64)
denom = np.zeros((NUM_TILES_1D, NUM_TILES_1D), dtype=np.int64)
uncert = np.zeros((NUM_TILES_1D, NUM_TILES_1D), dtype=np.float64)

compute_until(rngs, numer, denom, uncert, 1e-5)

In [ ]:
fig, ax, p = plot_pixels(uncert, dpi=80)
fig.colorbar(p, ax=ax, shrink=0.8, label="area uncertainty estimate of each tile")

None

In [ ]:
fig, ax, p = plot_pixels(denom, dpi=80)
fig.colorbar(p, ax=ax, shrink=0.8, label="number of points sampled each tile")

None

In [ ]:
final_value = (np.sum((numer / denom)) * width * height).item()
final_value

In [ ]:
CONFIDENCE_LEVEL = 0.05

confidence_interval_low, confidence_interval_high = confidence_interval(
    CONFIDENCE_LEVEL, numer, denom, width * height
)

final_uncertainty = combine_uncertaintes(
    confidence_interval_low, confidence_interval_high, denom
)
final_uncertainty